### PyFVCOM plotting tools examples

Here, we demonstrate plotting in three different dimensions: horizontal space, vertical space and time.

First we load some model output into an object which can be passed to a number of plotting objects. These objects have methods for plotting different aspects of the data.

For the horizontal plots, we plot the sea surface elevation across the model domain at a given time.
For the vertical plots, we take a transect through the water column temperature data and plot it.
For the time plots, we plot both a simple surface elevation time series and a time-varying water column temperature plot.


In [ ]:
import numpy as np
# Load an FVCOM model output and plot a surface.
from PyFVCOM.read import FileReader
from PyFVCOM.read import ncread
from PyFVCOM.plot import Plotter, Time, Depth
from PyFVCOM.tide import make_water_column
from cmocean import cm
import matplotlib.pyplot as plt
from pyproj import Proj
%matplotlib inline

## Check the FVCOM output NetCDF
`FileReader` instance takes long time; so better to first check dims and variables using `ncread`

In [ ]:
# Activate to chack NetCDF
'''
ds = ncread('tokyobay_futtsu_0001.nc')
#ds ### Activate to check
'''

In [ ]:
# Create an object which holds the model outputs. We're only loading
# surface elevation and temperature for the first 200 time steps.
# fvcom = FileReader('tokyobay_futtsu_0001.nc', dims={'time': [0,1]}, variables=['zeta', 'temp'])
fvcom = FileReader('tokyobay_futtsu_0001.nc', dims={'time': [0,1,2]}, variables=['zeta', 'temp'], zone=54)
#fvcom = ncread('tokyobay_futtsu_0001.nc')
#fvcom

In [ ]:
fvcom.dims

In [ ]:
fvcom.data.zeta[0,:]

## Parameters for Plotter instance
        ----------
        dataset : Dataset, PyFVCOM.read.FileReader
            netCDF4 Dataset or PyFVCOM.read.FileReader object.
        stations : 2D array, optional
            List of station coordinates to be plotted ([[lons], [lats]])
        extents : 1D array, optional
            Four element numpy array giving lon/lat limits as west, east, south, north (e.g. [-4.56, -3.76, 49.96,
            50.44])
        vmin : float, optional
            Lower bound to be used on colour bar (plot_field only).
        vmax : float, optional
            Upper bound to be used colour bar (plot_field only).
        mask : float, optional
            Mask out values < mask (plot_field only).
        res : string, optional
            Resolution to use when drawing Basemap object. If None, no coastline is plotted.
        fs : int, optional
            Font size to use when rendering plot text
        title : str, optional
            Title to use when creating the plot
        cmap : string, optional
            Colormap to use when shading field data (plot_field only).
        figure : Figure, optional
            Matplotlib figure object. A figure object is created if not
            provided.
        figsize : tuple(float), optional
            Figure size in cm. This is only used if a new Figure object is
            created.
        axes : Axes, optional
            Matplotlib Axes object. An Axes object is created if not
            provided.
        axis_position : 1D array, optional
            Array giving axis dimensions
        tick_inc : list, optional
            Add coordinate axes (i.e. lat/long) at the intervals specified in
            the list ([lon_spacing, lat_spacing]).
        cb_label : str, optional
            Set the colour bar label.
        extend : str, optional
            Set the colour bar extension ('neither', 'both', 'min', 'max').
            Defaults to 'neither').
        norm : matplotlib.colors.Normalize, optional
            Normalise the luminance to 0, 1. For example, use from matplotlib.colors.LogNorm to do log plots of fields.
        m : mpl_toolkits.basemap.Basemap, optional
            Pass a Basemap object rather than creating one on each invocation.
        cartesian : bool, optional
            Set to True to skip using Basemap and instead return a simple cartesian axis plot. Defaults to False
            (geographical coordinates).
        mapper : string, optional
            Set to 'basemap' to use Basemap for plotting or 'cartopy' for cartopy.
        coast : bool, optional
            Set to True to plot coastline. Default to True.
        bmargs : dict, optional
            Additional arguments to pass to Basemap.
        axis_labels : bool, optional
            Whether to annotate x and y axis with labels (defaults to Latitude and Longitude)
        bg_color: str, optional
            sets the figure background color. Defaults to gray
        line_width: float, optional
            sets line width. If missing, uses default in rcParams
### [pcolormesh](https://matplotlib.org/stable/gallery/images_contours_and_fields/pcolormesh_levels.html)

In [ ]:
# Make a plot of the surface elevation.
# fvcom.data.zeta[0,:] は zetaのスライス
#   最初の次元は時間で，FileReader instanceで抽出した要素を0始まりの連続したindexとして保持
# 課題
# ・海岸線はGSHHSを使用しており，精度が悪い．陸域の塗りつぶしや海岸線の扱いは引きつづき要検討
# ・日付や任意のテキストを簡単に入れられるようにする
# ・フォントサイズ，軸ラベルの変更
# ・datetimeで抽出できるようにする

plot = Plotter(fvcom,
               figsize=(20, 20),
               res='i',
               tick_inc=(4, 2),
               cb_label='{} ({})'.format(fvcom.atts.zeta.long_name,
                                         fvcom.atts.zeta.units),
               # [west, east, south, north]
               extents=[130, 145, 30, 40], vmin=-0.001, vmax=0.001,
               cmap=cm.balance)
plot.plot_field(fvcom.data.zeta[2, :])
plot.figure.savefig('zeta_h2d.png', dpi=600, bbox_inches='tight')

#plot.cmap
#plot.axes.set_title(fvcom.time.datetime[0].strftime('%Y-%m-%d %H:%M:%S'))

### `horizontal_transect_nodes(self, poitions)`
In `PyFVCOM/grid/_grid__.py` (L:1990)<br>
        positions : np.ndarray
            -Extract node IDs along a line defined by `positions` [[x1, y1], [x2, y2], ..., [xn, yn]].
            -Array of positions along which to sample the grid. Units are spherical decimal degrees.

In [ ]:
# Plot a temperature transect between two locations.
# 課題
# ・カラーバーの目盛りの数字を普通にしたい
# ・間隔を指定する，コンターにする．

#positions = np.array(((lon1, lat1), (lon2, lat2)))
positions = np.array(((138, 33), (139, 33)))
indices, distance = fvcom.horizontal_transect_nodes(positions)
# print(indices); print(distance)

plot = Depth(fvcom, figsize=(20, 9),
             cb_label='Temperature ({})'.format(fvcom.ds.variables['temp'].units),
             cmap=cm.thermal)
# fill_seabed makes the part of the plot below the seabed grey.
plot.plot_slice(distance / 1000,  # to kilometres from metres
                fvcom.grid.siglay_z[:, indices],
                fvcom.data.temp[2, :, indices],
                fill_seabed=True)
plot.axes.set_xlim(right=(distance / 1000).max())  # set the x-axis to the data range
plot.axes.set_xlabel('Distance (km)')
plot.axes.set_ylabel('Depth (m)')
# Save the figure.
plot.figure.savefig('temperature_profile.png', dpi=600, bbox_inches='tight')

## `Time.plotline()`

In [ ]:
# Do a time series at a specific location.
# 課題（plotできない）
# ・Time classはdatetimeが前提となっている
# ・x軸のself.timeは要datetimeのため，現在はNoneになっている，
gauge = (33, 139)  # a sample (lon, lat) position
index = fvcom.closest_node(gauge)
print(index)
print(fvcom.data.zeta[:,index])
time = Time(fvcom, figsize=(20, 9), title='{} at {}, {}'.format(fvcom.atts.zeta.long_name,
                                                                *gauge))
time.plot_line(fvcom.data.zeta[:, index], color='r')
time.axes.set_ylabel('{} ({})'.format(fvcom.atts.zeta.long_name,
                                      fvcom.atts.zeta.units))

In [ ]:
# Plot a depth-varying time profile through a water column
fvcom = FileReader('sample.nc', variables=['temp', 'zeta'], dims={'time': range(400), 'node': [5000]})
time = Time(fvcom, figsize=(20, 9), cb_label='{} ({})'.format(fvcom.atts.temp.long_name,
                                                              fvcom.atts.temp.units))
z = make_water_column(fvcom.data.zeta, fvcom.grid.h, fvcom.grid.siglay)
# fill_seabed makes the part of the plot below the seabed grey.
# We need to squeeze the data array since we've only extracted a single
# position.
time.plot_surface(z, np.squeeze(fvcom.data.temp), fill_seabed=True)
time.axes.set_ylabel('Depth (m)')